# Maloplošná zvláště chráněná území
Ukázka clusteringu na reálném příkladu - Hranice vyhlášených maloplošných zvláště chráněných území (národní přírodní rezervace, národní přírodní památky, přírodní rezervace, přírodní památky); vrstva obsahuje složené prvky (Multipart Features); © AOPK ČR. 

Celkem 2645 samostatných území složíme do různých clusterů, které potom budeme zobrazovat.

## Načtení dat

In [ ]:
###
# stahnete si soubor z teto URL
#! [ -f maloplosna_uzemi.zip ] || wget https://opendata.arcgis.com/datasets/91b1bb5621ae40a58dfddcc4550e147a_2.zip -O maloplosna_uzemi.zip

In [ ]:
###
import geopandas
import contextily as ctx
import numpy as np
import sklearn.cluster 
import sklearn.mixture 
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
###
gdf = geopandas.read_file("zip://maloplosna_uzemi.zip")
gdf

## Vizualizace dat
Zkusíme data vizualizovat

In [ ]:
# zobrazení GDF na mapě
gdf.plot(figsize=(12, 10))
plt.axis("off")

Vidíme, že velké množství rezervací zůstalo skryto, protože jsou velmi malé. Proto je transformujeme na body
## Transformace dat

In [ ]:
# Vytvoříme nový data frame gdf_c tak, ze
# - bude kopie gdf
# - bude obsahovat sloupec area (pozor, nutne pouzit EPSG:5514 - S-JTSK)
# - polyhony budou nahrazeny body (centroid)
# - cely dataframe bude zpet ve EPSG:3857 (WebMercator)

gdf_c = gdf.to_crs("EPSG:5514") # spravny system pro praci s velikostmi
gdf_c["area"] = gdf_c.area
gdf_c = gdf_c.set_geometry(gdf_c.centroid).to_crs(epsg=3857)
gdf_c

In [ ]:
# A tato data vykreslíme na mapě (přidat alpha)
ax = gdf_c.plot(figsize=(20, 20), alpha=0.3, color="tab:red")

## Vytváření clusterů
V mapě vidíme oblasti, kde je velké množství bodů přes sebe. Zkusíme tedy vytvořit rozumné množství clusterů (shluků), které budou pokrývat více bodů.

In [ ]:
# Prvním krokem je vytvoření matice X o rozměrech (2645, 2) obsahující souřadnice (x,y)
coords = np.dstack([gdf_c.geometry.x, gdf_c.geometry.y]).reshape(-1, 2)
coords  ###

Nyní použíjeme učení bez učitele - shlukovací metodu. Vzhledem k velkému množství bodů použijeme metodu __k-means__ implementovanou v třídě `sklearn.cluster.MiniBatchKMeans`. Vhodný počet clusterů ověříme experimentálně. Výsledné přiřazení máme v atributu `labels_` (viz dokumentace).

In [ ]:
db = sklearn.cluster.MiniBatchKMeans(n_clusters=280).fit(coords)
db.labels_

In [ ]:
# Vytvoříme gdf4 obsahující kopii gdf_c a přidaný sloupec cluster odpovídající labelu
gdf4 = gdf_c.copy()
gdf4["cluster"] = db.labels_
gdf4

In [ ]:
# spojeni dohromad0 (funkce dissolve - geograficky ekvivalent groupby)
# KOD agregujeme jako pocet (a přejmenujeme na cnt) a plochu sumujeme
gdf4 = gdf4.dissolve(by="cluster", aggfunc={"KOD": "count", "area": "sum"}).rename(columns=dict(KOD="cnt"))
gdf4

Všimněte si, že se nám pro geometrii vytvořily bodů geometrie typu `MULTIPOINT`.

In [ ]:
plt.figure(figsize=(20, 20)) ###
ax = plt.gca() ###
# zobrazíme oblasti pomocí convex_hull (pokrytí všech bodů)
gdf4.convex_hull.plot(ax=ax, alpha=0.4)
# a také všechny body, co jsme měli k dispozici
gdf_c.plot(ax=ax, color="tab:red", alpha=0.5)

ctx.add_basemap(ax, crs=gdf_c.crs.to_string(), source=ctx.providers.Stamen.Terrain, alpha=0.6) ###

## Reprezentativní body
Nyní musíme určit reprezentativní body. Mohli bychom použít čistě `centroid`. Ovšem shlukovací metoda nám rovnou určuje středy v proměnné `cluster_centers_`

In [ ]:
db.cluster_centers_ ###

In [ ]:
###
gdf_coords = geopandas.GeoDataFrame(geometry=geopandas.points_from_xy(db.cluster_centers_[:, 0], db.cluster_centers_[:, 1]))
gdf_coords

In [ ]:
gdf5 = gdf4.merge(gdf_coords, left_on="cluster", right_index=True).set_geometry("geometry_y")
gdf5

In [ ]:
# Zobrazíme graf tak, že velikost bodu bude odpovídat 
plt.figure(figsize=(20, 8)) ###
ax = plt.gca()  ###
gdf5.plot(ax=ax, markersize=gdf5["area"] / 1e5, column="cnt", legend=True)
ctx.add_basemap(ax, crs="epsg:3857", source=ctx.providers.OpenTopoMap, zoom=8, alpha=0.6) ###

In [ ]:
###
# pro srovnání půbodní oblasti
gdf.to_crs(epsg=3857).plot(figsize=(20, 8))